In [1]:
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

In [2]:
from rake_nltk import Rake
rake_nltk_var = Rake()
text = """cat and dog and lion are walking, in a thick and deep forest"""
rake_nltk_var.extract_keywords_from_text(text)
keyword_extracted = rake_nltk_var.get_ranked_phrases()
print(keyword_extracted)

['deep forest', 'walking', 'thick', 'lion', 'dog', 'cat']


In [3]:
# nltk.download("tag")

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))


# tokenized = sent_tokenize(text)

tagged = nltk.pos_tag(keyword_extracted)
print(tagged)

[('deep forest', 'JJS'), ('walking', 'VBG'), ('thick', 'JJ'), ('lion', 'NN'), ('dog', 'NN'), ('cat', 'NN')]


In [5]:
import json
import re
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')
def findMaxCosine(extNoun):
    maxDict = {}
    animals_adj = open("./animal_adj.json")
    ani_json = json.load(animals_adj)
    for i in ani_json:
        x = re.search(" " + extNoun + "$", i)
        if x:
            col1=[extNoun]
            col2=[i]
            vectors1 = model.encode(col1, convert_to_tensor=True)
            vectors2 = model.encode(col2, convert_to_tensor=True)
            cosine_scores = util.cos_sim(vectors1, vectors2)
            for j,(sent1,sent2) in enumerate(zip(col1,col2)):
                if cosine_scores[j][j]>=0.5:
                    maxDict[i] = cosine_scores[j][j]
    # print("For " + extNoun, " \n")
    maxCosine = []
    l = 5
    ind = 0
    for k, v in sorted(maxDict.items(), key=lambda item: item[1],reverse=True):
        if(ind >= l or ind >= len(maxDict)):
            break
        ind+=1
        maxCosine.append(k)
    return maxCosine
    
    
allAdj = {}
allText = []
k=0
for i,j in (tagged):
    if(j=='NN'):
        allAdj[i] = findMaxCosine(i)
allText.append([])
allText[0]=[text]
length=1
for k in allAdj:
    allText.append([])
    for v1 in allAdj[k]:
        for m in allText[length-1]:
            allText[length].append(re.sub(k,v1,m))
    length+=1
for i in allText[length-1]:
    print(i+"\n")

e:\FYP\bert_sentence_expander\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tiger cat and Bush dog and African lion are walking, in a thick and deep forest

Tiger cat and Bush dog and Mountain lion are walking, in a thick and deep forest

Tiger cat and Bush dog and Asian lion are walking, in a thick and deep forest

Tiger cat and Bush dog and Australian sea lion are walking, in a thick and deep forest

Tiger cat and Bush dog and Galapagos sea lion are walking, in a thick and deep forest

Tiger cat and Raccoon dog and African lion are walking, in a thick and deep forest

Tiger cat and Raccoon dog and Mountain lion are walking, in a thick and deep forest

Tiger cat and Raccoon dog and Asian lion are walking, in a thick and deep forest

Tiger cat and Raccoon dog and Australian sea lion are walking, in a thick and deep forest

Tiger cat and Raccoon dog and Galapagos sea lion are walking, in a thick and deep forest

Tiger cat and African wild dog and African lion are walking, in a thick and deep forest

Tiger cat and African wild dog and Mountain lion are walking, 

In [6]:
# nltk.download('stopwords')

In [7]:
# print(text)
def cosineTop10(allText1):
    maxDict1 = {}
    top10Sentences = []
    for i in allText1:
        col1=[i]
        col2=[text]
        vectors1 = model.encode(col1, convert_to_tensor=True)
        vectors2 = model.encode(col2, convert_to_tensor=True)
        cosine_scores = util.cos_sim(vectors1, vectors2)
        for j,(sent1,sent2) in enumerate(zip(col1,col2)):
            if cosine_scores[j][j]>=0.5:
                maxDict1[i] = cosine_scores[j][j]
    l1 = 10
    ind1 = 0
    for k, v in sorted(maxDict1.items(), key=lambda item: item[1],reverse=True):
        if(ind1 >= l1 or ind1 >= len(maxDict1)):
            break
        ind1+=1
        top10Sentences.append(k)
    return top10Sentences 
topSentences = cosineTop10(allText[length-1])
    

In [8]:
for i in topSentences:
    print(i,end = "\n")

Tiger cat and Bush dog and Mountain lion are walking, in a thick and deep forest
Tiger cat and Bush dog and African lion are walking, in a thick and deep forest
Tiger cat and Raccoon dog and African lion are walking, in a thick and deep forest
Jungle cat and Bush dog and Mountain lion are walking, in a thick and deep forest
Tiger cat and African wild dog and Mountain lion are walking, in a thick and deep forest
Tiger cat and African wild dog and African lion are walking, in a thick and deep forest
Tiger cat and Bush dog and Asian lion are walking, in a thick and deep forest
Tiger cat and Raccoon dog and Mountain lion are walking, in a thick and deep forest
Jungle cat and Bush dog and Asian lion are walking, in a thick and deep forest
Tiger cat and African wild dog and Asian lion are walking, in a thick and deep forest


Dependecy identification and Tree Generation

In [18]:
import spacy
nlp = spacy.load("en_core_web_sm")
sentence = "One cat and two dog are in the house"
doc = nlp(sentence)
for token in doc:
    print(f"{token.text}: {token.dep_} -> {token.head.text}")

One: nummod -> cat
cat: nsubj -> are
and: cc -> cat
two: nummod -> dog
dog: conj -> cat
are: ROOT -> are
in: prep -> are
the: det -> house
house: pobj -> in


In [31]:
import spacy
import networkx as nx
import matplotlib.pyplot as plt

nlp = spacy.load("en_core_web_sm")

sentence = "cat in the house and drinking milk which is stolen by other cats"

doc = nlp(sentence)

G = nx.DiGraph()

dep_labels = {
    "acl": "Clausal modifier of noun",
    "acomp": "Adjectival complement",
    "advcl": "Adverbial clause modifier",
    "advmod": "Adverbial modifier",
    "agent": "Agent of a passive construction",

}

for token in doc:
    G.add_node(token.i, label=token.text, pos=token.pos_, dep=token.dep_, head=token.head.i)
    if token.i != token.head.i:  # Skip self-loops
        G.add_edge(token.head.i, token.i, label=dep_labels.get(token.dep_, token.dep_))

pos = nx.spring_layout(G, seed=42)

incoming_nodes = {n for n in G.nodes() if G.in_degree(n) > 0}
outgoing_nodes = {n for n in G.nodes() if G.out_degree(n) > 0}

incoming_labels = {i: data['label'] for i, data in G.nodes(data=True) if i in incoming_nodes}
nx.draw_networkx_nodes(G, pos, nodelist=incoming_nodes, node_color='lightblue', node_size=800)
nx.draw_networkx_labels(G, pos, labels=incoming_labels, font_size=10, font_color='black')

outgoing_labels = {i: data['label'] for i, data in G.nodes(data=True) if i in outgoing_nodes}
nx.draw_networkx_nodes(G, pos, nodelist=outgoing_nodes, node_color='lightgreen', node_size=800)
nx.draw_networkx_labels(G, pos, labels=outgoing_labels, font_size=10, font_color='black')

edge_labels = {(i, j): data['label'] for i, j, data in G.edges(data=True)}
nx.draw_networkx_edges(G, pos, edge_color='gray', arrowsize=20)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, font_color='red')

plt.axis('off')
plt.show()


TypeError: complete_graph() missing 1 required positional argument: 'n'

In [42]:
#Algorithm implementation
import json
class Node:
    def __init__(self,val,adj):
        self.val = val #string
        self.adj = adj #[]
    def getVal(self):
        return self.val

class GoT:
    def __init__(self,arr): # arr contains nsubj, dobj
        
        self.children = []
    def createGraph(self):
        for i in self.data_json:
            n = Node(i["name"])
            self.children.append(Node("adjectives"))
            self.children.append(Node("habitats"))
                    
                

got = GoT(["Tiger","lion"])
got.createGraph()

Frog
{'name': 'Frog', 'adjectives': ['colorful', 'poisonous', 'tiny', 'vibrant', 'striking', 'tropical'], 'habitats': ['rainforests', 'jungles', 'tropical forests']}
{'name': 'Eagle', 'adjectives': ['majestic', 'imposing', 'predatory', 'large', 'powerful', 'elegant'], 'habitats': ['tropical rainforests', 'dense jungles', 'remote forested regions']}
{'name': 'Jaguar', 'adjectives': ['powerful', 'ferocious', 'spotted', 'solitary', 'elusive', 'carnivorous'], 'habitats': ['rainforests', 'swamps', 'grasslands']}
{'name': 'Toucan', 'adjectives': ['colorful', 'tropical', 'vibrant', 'beak-shaped', 'noisy', 'fruit-eating'], 'habitats': ['rainforests', 'tropical forests', 'jungles']}
{'name': 'Sloth', 'adjectives': ['slow-moving', 'tree-dwelling', 'peaceful', 'herbivorous', 'folivorous', 'sleepy'], 'habitats': ['tropical rainforests', 'canopy', 'forested regions']}
{'name': 'Orangutan', 'adjectives': ['intelligent', 'solitary', 'arboreal', 'ginger-haired', 'strong', 'endangered'], 'habitats': ['

In [87]:
class Node:
    def __init__(self, name, node_type):
        self.name = name
        self.node_type = node_type
        self.children = []
    def add_child(self, child):
        self.children.append(child)
    def __str__(self) -> str:
        return f"{self.name} - ({self.node_type})"
   
class Graph:
    def __init__(self):
        self.nodes = []

    def add_node(self, node):
        self.nodes.append(node)

    def add_edge(self, parent, child):
        parent.add_child(child)

    def get_children(self, node):
        return node.children
    def findNode(self,animalName):
        for i in self.nodes:
            if(i.node_type == 'animal' and i.name == animalName):
                temp =  self.get_children(i)
                return temp
        return []
# Create a graph
graph = Graph()
data = open('./data.json')

data_json = json.load(data)

data_json = data_json['animals']

root = Node("animal", "root")
graph.add_node(root)
for i in range(len(data_json)):
    n1 = Node(data_json[i]['name'],"animal")
    graph.add_node(n1)
    for adj in data_json[i]["adjectives"]:
        adjective_node = Node(adj, "adjective")
        graph.add_node(adjective_node)
        graph.add_edge(n1, adjective_node)

    for habitat in data_json[i]["habitats"]:
        habitat_node = Node(habitat, "habitat")
        graph.add_node(habitat_node)
        graph.add_edge(n1, habitat_node)

testAnimal = "Giraffe"
cdren = graph.findNode(testAnimal)
if(cdren != []):
    for i in cdren:
        print(i)
else:
    print("Not Found")

tall - (adjective)
graceful - (adjective)
herbivorous - (adjective)
spotted - (adjective)
gentle - (adjective)
long-necked - (adjective)
African savannah - (habitat)
woodlands - (habitat)
grasslands - (habitat)
